# 8. Calculate

Make sure dependencies and env are as it should be

ppp_env_x86.yaml / soyadapt_env_x86

In [3]:
import sys
import os
import subprocess

from pgpipe import four_gamete, vcf_split_pysam, model_creator, vcf_to_ima, vcf_filter, vcf_calc, vcf_sampler, vcf_phase, stat_sampler, vcf_split, vcf_utilities, vcf_to_treemix
from pgpipe.logging_module import initLogger
from pgpipe.informative_loci_filter import filter_bed_regions
from pgpipe.subtract_bed import filter_stat
import pysam

print ("Imports complete")

Imports complete


In [6]:
#define project name
#project_name = 'projectname'
project_name = 'soyadapt_data_analysis' # [ ] 

import os.path
path = '/Users/josephinelovemachine'
full_path = os.path.expanduser(path)
print (full_path)
# set home directory
# ~/<projectname>/
home_dir = os.path.join( full_path, project_name)
print (home_dir)
print ("home directory defined")


/Users/josephinelovemachine
/Users/josephinelovemachine/soyadapt_data_analysis
home directory defined


In [7]:
#Set up directories and filepaths, run on all restarts
config_dir = home_dir + '/config/'
results_dir = home_dir + '/results/'
workflow_dir = home_dir + '/workflow/' 
notebook_dir = workflow_dir + '/notebooks/' #WHE RE THIS NOTEBOOK SHOULD BE [X]
# my working dir is where my data is ~/project_name/resources/
work_dir = home_dir + '/resources/'
#start data files:'
data_dir= work_dir +'start_data/' # my main_vcf_data is here
#generated data files:
data_generated = work_dir + 'generated_data/'
#generated data files:
logfile = data_generated+'logfiles/'

#Define start data name and location:
# data names are set to data _ name _ *** (e.g. data_name_snp)
# data locations are set to data _ *** _ datatype (e.g. data_snp_vcf)
#Which data do you want to use?
data_name_snp = 'soysnp50k' 
data_snp_vcf = data_dir+'soysnp50k.vcf.gz' 
data_name_test = 'test_data' 
data_test_vcf = data_dir+'test_data.vcf.gz' 
data_name_intersection = 'intersection_data'
data_intersection_vcf = data_dir+'intersection_data.vcf.gz' 
data_name_imputed = 'imputed_data'
data_imputed_vcf = data_dir+'imputed_data.vcf.gz' 

# ADDED Metadata from start
#chromosomes list
list_vcf_name_chr = data_dir+'soybean_resequencing_MQ30_list_vcf_name_chr'
# string of all 20 chromosomes named chr
chr_all = data_dir+'chr.txt' 
# The 50k snps list 
list_snp_locations_50k = data_dir+'soybean_resequencing_MQ30_ist_snp_locations_50k'
# metadata list:
list_vcf_name_country = data_dir+'soybean_resequencing_MQ30_list_vcf_name_country'
list_vcf_name_maturitygroup = data_dir+'soybean_resequencing_MQ30_list_vcf_name_maturitygroup'
#list of accessions + population(CCA/SA/Founders) + country + maturity group
list_vcf_name_collected = data_dir+'soybean_resequencing_MQ30_list_vcf_name_collected'
#data files lists so they are named accordingly when generated or added to the project:
list_vcf_name_sa= data_dir+'soybean_resequencing_MQ30_list_vcf_name_sa'
list_vcf_name_cca= data_dir+'soybean_resequencing_MQ30_list_vcf_name_cca'
list_vcf_name_founders= data_dir+'soybean_resequencing_MQ30_list_vcf_name_founders'
stat_file_name= data_generated+'soybean_resequencing_MQ30_stat_file'

loci=50000
# print check directories and files
print(work_dir)
print(data_dir)
print(data_generated)


/Users/josephinelovemachine/soyadapt_data_analysis/resources/
/Users/josephinelovemachine/soyadapt_data_analysis/resources/start_data/
/Users/josephinelovemachine/soyadapt_data_analysis/resources/generated_data/


In [ ]:
#Set up directory structure, only needs to be run once
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
if not os.path.exists(workflow_dir):
    os.makedirs(workflow_dir)
if not os.path.exists(notebook_dir):
    os.makedirs(notebook_dir)            
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
if not os.path.exists(work_dir+'generated_data/'):
    os.makedirs(work_dir+'generated_data/')
if not os.path.exists(data_dir):
    os.makedirs(work_dir+'start_data/')
if not os.path.exists(data_generated):
    os.makedirs(data_generated)
if not os.path.exists(data_generated+'logfiles/'):
    os.makedirs(data_generated+'logfiles/')
# add files for the project that dont need variables allocated and workflow access
if not os.path.exists(workflow_dir+'envs/'): # envs for each workflow
    os.makedirs(workflow_dir+'envs/')
if not os.path.exists(workflow_dir+'scripts/'): # scripts for the project like R scripts
    os.makedirs(workflow_dir+'scripts/')
if not os.path.exists(workflow_dir+'notebooks/'): # this notebook is here! move now! 
    os.makedirs(workflow_dir+'notebooks/')
if not os.path.exists(workflow_dir+'repots/'): # if not in logfile, then here
    os.makedirs(workflow_dir+'reports/')

print('move this notebook now to the notebook directory')

cant get it to work right now, for simplicities sake:
from PPP
Example 1: Simple Model

A basic model only require a single population (pop) with a single in individual (ind). Only three commands are required:

    Create and name a model: --model 1Pop
    Assign a pop to a model: --model-pop 1Pop Paniscus
    Assign an ind to a pop: --pop-ind Paniscus Pan_paniscus-9731_LB502


In [11]:
model_creator.run(['--model', '1Pop', '--model-pop', '1Pop', 'Pop_one', '--pop-ind', 'Pop_one', '387A', '--model-pop', '1Pop', 'Pop_two', '--pop-ind', 'Pop_two', '406A', '--model-tree-file', '1Pop', data_dir+'tree.txt'])

In [13]:
#Calculates f_st statistics across genome
vcf_calc.run(['--vcf', main_vcf_data, '--out', stat_file_name, '--calc-statistic', 'weir-fst', '--statistic-window-size', '10000', '--statistic-window-step', '10000', '--overwrite', '--vcf-pop', 'Pop_one', '--vcf-pop', 'Pop_two'])
print("Stat calculation complete")

usage: ipykernel_launcher.py [-h] --vcf VCF [--model-file MODEL_FILE]
                             [--model MODEL]
                             [--pop-file POP_FILE [POP_FILE ...]] [--out OUT]
                             [--out-prefix OUT_PREFIX] [--out-dir OUT_DIR]
                             [--overwrite] --calc-statistic {weir-fst,
                             windowed-weir-fst, TajimaD, site-pi, window-pi,
                             freq, het-fit, het-fis, hardy-weinberg}
                             [--statistic-window-size STATISTIC_WINDOW_SIZE]
                             [--statistic-window-step STATISTIC_WINDOW_STEP]
                             [--filter-include-positions FILTER_INCLUDE_POSITIONS]
                             [--filter-exclude-positions FILTER_EXCLUDE_POSITIONS]
                             [--filter-include-bed FILTER_INCLUDE_BED]
                             [--filter-exclude-bed FILTER_EXCLUDE_BED]
                             [--filter-include-indv F

SystemExit: 2

/opt/homebrew/anaconda3/envs/ppp_env_x86/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3560: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
vcf_calc.run([--vcf examples/files/merged_chr1_10000.vcf.gz --model-file examples/files/input.model --model 1Pop --calc-statistic windowed-weir-fst --statistic-window-size 10000 --statistic-window-step 10000])

SyntaxError: invalid syntax (662998507.py, line 1)